<a href="https://colab.research.google.com/github/priyanshgupta1998/Natural-language-processing-NLP-/blob/master/word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
files.upload()
# Next, install the Kaggle API client.
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [10]:
!kaggle competitions list -s nlp

ref                    deadline             category            reward  teamCount  userHasEntered  
---------------------  -------------------  ---------------  ---------  ---------  --------------  
word2vec-nlp-tutorial  2015-06-30 23:59:00  Getting Started  Knowledge        578           False  


In [21]:
!kaggle competitions files word2vec-nlp-tutorial

name                         size  creationDate         
--------------------------  -----  -------------------  
sampleSubmission.csv        276KB  2014-07-21 22:24:56  
unlabeledTrainData.tsv.zip   26MB  2014-08-05 23:31:46  
testData.tsv.zip             13MB  2014-08-05 23:31:52  
labeledTrainData.tsv.zip     13MB  2014-08-05 23:32:11  


In [0]:
!kaggle competitions download -c word2vec-nlp-tutorial

In [0]:
from importlib import reload
import sys
from imp import reload
import warnings
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

In [14]:
!unzip labeledTrainData.tsv.zip

Archive:  labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    


In [19]:

import pandas as pd

df1 = pd.read_csv('labeledTrainData.tsv', delimiter="\t")
df1 = df1.drop(['id'], axis=1)
print(df1.shape)
df1.head()

(25000, 2)


,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [20]:
!unzip unlabeledTrainData.tsv.zip

Archive:  unlabeledTrainData.tsv.zip
  inflating: unlabeledTrainData.tsv  


https://www.kaggle.com/utathya/imdb-review-dataset          
https://www.kaggle.com/c/word2vec-nlp-tutorial

In [22]:
!kaggle datasets download -d utathya/imdb-review-dataset

 81% 41.0M/50.5M [00:00<00:00, 46.1MB/s]
100% 50.5M/50.5M [00:00<00:00, 85.7MB/s]


In [23]:
!unzip imdb-review-dataset.zip

Archive:  imdb-review-dataset.zip
  inflating: imdb_master.csv         


In [24]:
df2 = pd.read_csv('imdb_master.csv',encoding="latin-1")
df2.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [25]:
df2 = df2.drop(['Unnamed: 0','type','file'],axis=1)
df2.columns = ["review","sentiment"]
df2.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [26]:
df2.sentiment.value_counts()

unsup    50000
neg      25000
pos      25000
Name: sentiment, dtype: int64

In [27]:
df2 = df2[df2.sentiment != 'unsup']   # it should be either positive or negative
df2['sentiment'] = df2['sentiment'].map({'pos': 1, 'neg': 0})
df2.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0


In [29]:
df = pd.concat([df1, df2]).reset_index(drop=True)
df.head()

,review,sentiment
0,With all this stuff going down at the moment w...,1
1,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,The film starts with a manager (Nicholas Bell)...,0
3,It must be assumed that those who praised this...,0
4,Superbly trashy and wondrously unpretentious 8...,1


In [33]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [34]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()    # to find the root word of the word


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))
print(df.shape)
df.head()

(75000, 3)


,review,sentiment,Processed_Reviews
0,With all this stuff going down at the moment w...,1,stuff go moment mj ive start listen music watc...
1,"\The Classic War of the Worlds\"" by Timothy Hi...",1,classic war world timothy hines entertain film...
2,The film starts with a manager (Nicholas Bell)...,0,film start manager nicholas bell give welcome ...
3,It must be assumed that those who praised this...,0,must assume praise film greatest film opera ev...
4,Superbly trashy and wondrously unpretentious 8...,1,superbly trashy wondrously unpretentious 80 ex...


In [38]:
pd.DataFrame(df.Processed_Reviews.apply(lambda x: len(x.split(" ")))).head(10)    # total number of words in the text(Processed_Reviews)

,Processed_Reviews
0,232
1,82
2,249
3,207
4,217
5,45
6,59
7,72
8,101
9,26


In [37]:
df.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

128.51009333333334

#use deep learning (RNN with tensorflow) to train our model

In [39]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [0]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(df['Processed_Reviews'])

In [50]:
tokenizer

In [56]:
print(len(list_tokenized_train))
print(len(list_tokenized_train[0]),  len(list_tokenized_train[1]))
      
s=0
for i in range(len(list_tokenized_train)):
    s += len(list_tokenized_train[i])
s

75000
203 72


8465664

In [53]:
maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
print(len(X_t))
print(len(X_t[0]) , len(X_t[1]))

75000
130 130


In [57]:
y = df['sentiment']
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))

W0615 09:58:45.447781 139964929185664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0615 09:58:45.494542 139964929185664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0615 09:58:45.501290 139964929185664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0615 09:58:46.116928 139964929185664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0615 09:58:46.123656 

In [58]:

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

W0615 09:59:05.769182 139964929185664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0615 09:59:05.795037 139964929185664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0615 09:59:05.801277 139964929185664 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         768000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 64)          41216     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 810,537
Trainable params: 810,537
Non-trainable params: 0
_________________________________________________________________
None

In [59]:
batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 60000 samples, validate on 15000 samples
Epoch 1/3
60000/60000 [==============================] - 204s 3ms/step - loss: 0.3351 - acc: 0.8466 - val_loss: 0.2248 - val_acc: 0.9156
Epoch 2/3
60000/60000 [==============================] - 199s 3ms/step - loss: 0.2165 - acc: 0.9157 - val_loss: 0.2683 - val_acc: 0.8931
Epoch 3/3
60000/60000 [==============================] - 200s 3ms/step - loss: 0.1680 - acc: 0.9380 - val_loss: 0.1211 - val_acc: 0.9618


In [61]:
!unzip testData.tsv.zip

Archive:  testData.tsv.zip
  inflating: testData.tsv            


In [62]:
df_test=pd.read_csv("testData.tsv",header=0, delimiter="\t", quoting=3)
print(df_test.shape)
df_test.head()

(25000, 2)


,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [63]:
def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df_test["review"] = df_test.review.apply(lambda x: clean_text(x))
df_test.head()

,id,review
0,"""12311_10""",naturally film main theme mortality nostalgia ...
1,"""8348_2""",movie disaster within disaster film full great...
2,"""5828_4""",movie kid saw tonight child love one point kid...
3,"""7186_2""",afraid dark leave impression several different...
4,"""12128_7""",accurate depiction small time mob life film ne...


In [64]:

df_test["sentiment"] = df_test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
df_test.head()

,id,review,sentiment
0,"""12311_10""",naturally film main theme mortality nostalgia ...,1
1,"""8348_2""",movie disaster within disaster film full great...,0
2,"""5828_4""",movie kid saw tonight child love one point kid...,0
3,"""7186_2""",afraid dark leave impression several different...,0
4,"""12128_7""",accurate depiction small time mob life film ne...,1


In [0]:

y_test = df_test["sentiment"]
list_sentences_test = df_test["review"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [66]:
prediction = model.predict(X_te)
print(len(prediction))
y_pred = (prediction > 0.5)
print(len(y_pred))

25000
25000


>>>![alt text](https://cdncontribute.geeksforgeeks.org/wp-content/uploads/Confusion_Matrix6.png)

In [67]:
from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)  #A confusion matrix is a table that is often used to describe the performance of a classification model.

F1-score: 0.9578410841992968
Confusion matrix:


array([[11812,   379],
       [  688, 12121]])

###the embedding layer in Keras can be used when we want to create the embeddings to embed higher dimensional data into lower dimensional vector space.

In [0]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#nltk
import nltk
#stop-words
from nltk.corpus import stopwords
stop_words=set(nltk.corpus.stopwords.words('english'))

# tokenizing
from nltk import word_tokenize,sent_tokenize

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input
from keras.models import Model

In [0]:
sample_text_1="bitty bought a bit of butter"
sample_text_2="but the bit of butter was a bit bitter"
sample_text_3="so she bought some better butter to make the bitter butter better"

corp=[sample_text_1,sample_text_2,sample_text_3]
no_docs=len(corp)
print(no_docs)

vocab_size=50
encod_corp=[]
for i,doc in enumerate(corp):
    encod_corp.append(one_hot(doc,vocab_size))
    print("The encoding for document",i+1," is : ",one_hot(doc) , '     ',doc)

In [76]:
# Generate dummy data
import numpy as np
data = np.random.random((2, 5))
labels = np.random.randint(3, size=(2, 3))

data, labels

(array([[0.37582643, 0.64361163, 0.05558382, 0.87132103, 0.75420242],
        [0.46744563, 0.28388764, 0.09329311, 0.7612881 , 0.40988677]]),
 array([[1, 2, 0],
        [1, 1, 2]]))

to_categorical is used to fill the array with the values either 0 or 1.  it is filled based on their position in the document or list . so , if we have 4 elements in our list then to_categorical() will create 4 different lists in that document (single) .      
But length of the list can't be greater than , num_classes parameter.

In [89]:
from keras.utils import to_categorical, normalize

to_categorical(labels, num_classes=4)

array([[[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)

In [81]:
to_categorical(labels, num_classes=3)

array([[[0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.]],

       [[0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]], dtype=float32)

In [82]:
to_categorical(labels, num_classes=6)

array([[[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.]]], dtype=float32)

In [87]:
normalize(data, order=1)     #Normalize is used to bring all the values within a range ( mostly between 0 and 1)

array([[0.37582643, 0.64361163, 0.05558382, 0.87132103, 0.75420242],
       [0.46744563, 0.28388764, 0.09329311, 0.7612881 , 0.40988677]])

`In keras , padded_sequences is used to fillup the blank places with zero. I mean to say , In this case all the lists  will have length equal to longest list. and their remaing places will be filled with 0. But their actual elements  will be filled in the list from the back side(from last position of  the list .)`

In [0]:
sequences = [
    [1,2,4,4],
    [3],
    [5,6,4,2,1,7,7,4,3],
    [3,3,4,3,2]
]

In [92]:

from keras.preprocessing.sequence import pad_sequences

padded_sequences = pad_sequences(sequences, 
                                 maxlen=None, 
                                 dtype='int32',
                                 padding='pre', 
                                 truncating='post', 
                                 value=0.)

padded_sequences

array([[0, 0, 0, 0, 0, 1, 2, 4, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 3],
       [5, 6, 4, 2, 1, 7, 7, 4, 3],
       [0, 0, 0, 0, 3, 3, 4, 3, 2]], dtype=int32)

**pad_sequences:** Transform a list of num_samples sequences (lists of scalars) into a 2D Numpy array of shape (num_samples, num_timesteps). num_timesteps is either the maxlen argument if provided, or the length of the longest sequence otherwise. Sequences that are shorter than num_timesteps are padded with value at the end. Sequences longer than num_timesteps are truncated so that it fits the desired length. Position where padding or truncation happens is determined by padding or truncating, respectively.


**skipgrams:** Transforms a sequence of word indexes (list of int) into couples of the form:
(word, word in the same window), with label 1 (positive samples).
(word, random word from the vocabulary), with label 0 (negative samples).

In [99]:
from keras.preprocessing.sequence import skipgrams
print('padded_sequences is : ' , padded_sequences[0],'\n\n\n')

grams = skipgrams(padded_sequences[0], 
                  vocabulary_size=8,
                  window_size=1, 
                  negative_samples=1.,
                  shuffle=True,
                  categorical=False)

grams   # it is random (arbitrary pair )

padded_sequences is :  [0 0 0 0 0 1 2 4 4] 





([[4, 1],
  [2, 4],
  [1, 4],
  [2, 1],
  [4, 4],
  [4, 1],
  [2, 7],
  [4, 2],
  [1, 2],
  [2, 3],
  [4, 7],
  [4, 4]],
 [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1])

In [0]:
text ="""
    My name is Nathaniel.
    I like data science.
    Let's do deep learning.
    Keras, my fave lib.
    """

In [102]:
from keras.preprocessing.text import text_to_word_sequence

words = text_to_word_sequence(text, lower=True, split=" ")

print(words)

['my', 'name', 'is', 'nathaniel', 'i', 'like', 'data', 'science', "let's", 'do', 'deep', 'learning', 'keras', 'my', 'fave', 'lib']


In [103]:
# we can change the filter chars too
print(text_to_word_sequence(text, filters="'", lower=True, split=" "))

['\n', 'my', 'name', 'is', 'nathaniel.\n', 'i', 'like', 'data', 'science.\n', 'let', 's', 'do', 'deep', 'learning.\n', 'keras,', 'my', 'fave', 'lib.\n']


In [112]:
from keras.preprocessing.text import one_hot
one_hot(text, n=8, lower=True, split=" ")

[3, 5, 6, 6, 5, 7, 5, 6, 2, 3, 3, 5, 5, 3, 6, 7]

In [0]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=None, lower=True, split=" ")

In [0]:
# tokenizer.fit_on_sequences
tokenizer.fit_on_texts([text])

In [114]:
tokenizer.texts_to_sequences([text])

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 14, 15]]

In [115]:
tokenizer.texts_to_matrix([text], 'count')

array([[0., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [116]:
tokenizer.texts_to_matrix(['Data Science is fun'], 'count')

array([[0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.]])